# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that we've discussed some regularization, initialization and optimization techniques, its time to synthesize those concepts into a cohesive modelling pipeline.  

With this pipeline, yoiu will not only fit an initial model but will also attempt to set various hyperparameters for regularization techniques. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modelling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. We've discussed the bias variance tradeoff, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Implement a K-folds cross validation modelling pipeline
* Apply normalization as a preprocessing technique
* Apply regularization techniques to improve your model's generalization
* Choose an appropriate optimization strategy 

## Loading the Data

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras import models, optimizers, initializers, layers
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler

sns.set_style('darkgrid')

In [15]:
#Your code here; load and preview the dataset
df = pd.read_csv('loan_final.csv')
df.dropna(inplace = True)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41394 entries, 0 to 42516
Data columns (total 16 columns):
loan_amnt              41394 non-null float64
funded_amnt_inv        41394 non-null float64
term                   41394 non-null object
int_rate               41394 non-null object
installment            41394 non-null float64
grade                  41394 non-null object
emp_length             41394 non-null object
home_ownership         41394 non-null object
annual_inc             41394 non-null float64
verification_status    41394 non-null object
loan_status            41394 non-null object
purpose                41394 non-null object
addr_state             41394 non-null object
total_acc              41394 non-null float64
total_pymnt            41394 non-null float64
application_type       41394 non-null object
dtypes: float64(6), object(10)
memory usage: 5.4+ MB


,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


## Defining the Problem

Set up the problem by defining X and Y. 

For this problem use the following variables for X:
* loan_amnt
* home_ownership
* funded_amnt_inv
* verification_status
* emp_length
* installment
* annual_inc

Be sure to use dummy variables for categorical variables and to normalize numerical quanitities. Be sure to also remove any rows with null data.  

For Y, we are looking to build a model to predict the total payment received for a loan.

In [16]:
# Your code here; appropriately define X and Y using dummy variables and normalization for preprocessing.
numerical = df[['loan_amnt','funded_amnt_inv','installment','annual_inc']]
categorical = df[['home_ownership', 'verification_status', 'emp_length']]

# Scale the numerical data
scaler = StandardScaler()
num_scaled = pd.DataFrame(scaler.fit_transform(numerical), columns = numerical.columns)

# One-hot Encode the Categorical Data
ohc = pd.get_dummies(categorical)

# Merge dataframes then preview
X = num_scaled.join(ohc)
X.info()
X.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41394 entries, 0 to 41393
Data columns (total 23 columns):
loan_amnt                              41394 non-null float64
funded_amnt_inv                        41394 non-null float64
installment                            41394 non-null float64
annual_inc                             41394 non-null float64
home_ownership_MORTGAGE                40279 non-null float64
home_ownership_NONE                    40279 non-null float64
home_ownership_OTHER                   40279 non-null float64
home_ownership_OWN                     40279 non-null float64
home_ownership_RENT                    40279 non-null float64
verification_status_Not Verified       40279 non-null float64
verification_status_Source Verified    40279 non-null float64
verification_status_Verified           40279 non-null float64
emp_length_1 year                      40279 non-null float64
emp_length_10+ years                   40279 non-null float64
emp_length_2 years     

,loan_amnt,funded_amnt_inv,installment,annual_inc,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
0,-0.831064,-0.731196,-0.774545,-0.708860,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.168176,-1.077707,-1.267125,-0.615909,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-1.181661,-1.091707,-1.150003,-0.890856,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.156841,-0.027675,0.068920,-0.318468,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.100754,-1.007705,-1.229072,0.158677,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
data = df.total_pymnt.values.reshape(-1,1)
y_scaled = scaler.fit_transform(data)
y = pd.DataFrame(y_scaled, columns = ['total_pymnt'])
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41394 entries, 0 to 41393
Data columns (total 1 columns):
total_pymnt    41394 non-null float64
dtypes: float64(1)
memory usage: 323.5 KB


In [49]:
# Must merge and drop NaNs since some were introduced when performing One-Hot Encoding.
Xy = X.join(y)
Xy.dropna(inplace = True)
X = Xy.drop('total_pymnt', axis = 1)
y = Xy.total_pymnt

In [50]:
# Review data again
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40279 entries, 0 to 41393
Data columns (total 23 columns):
loan_amnt                              40279 non-null float64
funded_amnt_inv                        40279 non-null float64
installment                            40279 non-null float64
annual_inc                             40279 non-null float64
home_ownership_MORTGAGE                40279 non-null float64
home_ownership_NONE                    40279 non-null float64
home_ownership_OTHER                   40279 non-null float64
home_ownership_OWN                     40279 non-null float64
home_ownership_RENT                    40279 non-null float64
verification_status_Not Verified       40279 non-null float64
verification_status_Source Verified    40279 non-null float64
verification_status_Verified           40279 non-null float64
emp_length_1 year                      40279 non-null float64
emp_length_10+ years                   40279 non-null float64
emp_length_2 years     

In [52]:
y.describe()

count    40279.000000
mean        -0.001686
std          0.999404
min         -1.329412
25%         -0.723902
50%         -0.256663
75%          0.481618
max          5.135071
Name: total_pymnt, dtype: float64

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modelling decisions. As such, pull out a sample of 10% of the total available data. For consistency of results, use random seed 123. 

In [53]:
# Your code here; generate a hold out test set for final model evaluation. Use random seed 123.
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.10)

## Defining a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, write a function that takes in the remaining data and performs k-folds cross validation given a model object. Be sure your function returns performance metrics regarding the training and validation sets.

In [75]:
import time

In [79]:
#Your code here; define a function to evaluate a model object using K folds cross validation.

def k_folds(features_train, labels_train, model_obj, k=10, n_epochs=100):
    # Make folds
    iterator = KFold(n_splits = k, shuffle = True).split(features_train)
    
    # Initialize data storage
    mse_list = []
    validation_scores = []
    
    for i,(data_i, val_i) in enumerate(iterator):
        start = time.time()
        x_data = features_train.iloc[data_i]
        y_data = labels_train.iloc[data_i]
        x_val = features_train.iloc[val_i]
        y_val = labels_train.iloc[val_i]
        
        hist = model_obj.fit(x_data, y_data, 
                             epochs = n_epochs, 
                             verbose = 0, 
                             batch_size = 32, 
                             validation_data = (x_val, y_val))
        
        mse = hist.history['mse'][-1]
        mse_list.append(mse)
        
        validation_score = model_obj.evaluate(x_val, y_val)
        validation_scores.append(validation_score)
        
        
        delta = time.time()-start
        print('Finished Fold {0} in {1:.2f} seconds'.format(i, delta))
        
    print('Mean MSE: {0:.2f}\t Std MSE: {1:.2f}'.format(np.mean(mse_list), np.std(mse_list)))
    print('Mean Val: {0:.2f}\t Std Val: {1:.2f}'.format(np.mean(validation_scores), np.std(validation_scores)))

## Building a Baseline Model

Here, it is also important to define your evaluation metric that you will look to optimize while tuning the model.   

In general, model training to optimize this metric may consist of using a validation and test set if data is plentiful, or k-folds cross-validation if data is limited. We set up a k-folds cross-validation for this task since the dataset is not overly large.  

Build an initial sequential model with 2 hidden relu layers. The first should have 7 hidden units, and the second 10 hidden units. Finally, add a third layer with a linear activation function to output our predictions for the total loan payment. 

In [71]:
#Your code here; define and compile an initial model as described
model = models.Sequential()
model.add(layers.Dense(units = 7, activation = 'relu', input_dim = 23))
model.add(layers.Dense(units = 10, activation = 'relu'))
model.add(layers.Dense(units = 1, activation = 'linear'))
model.compile(optimizer = 'sgd', loss = 'mse', metrics = ['mse'])

## Evaluating the Baseline Model with K-Folds Cross Validation

Use your k-folds function to evaluate the baseline model.  

Note: This code block is likely to take 10-20 minutes to run depending on the specs on your computer.
Because of time dependencies, it can be interesting to begin timing these operations for future reference.

Here's a simple little recipe to achieve this:
```
import time
import datetime

now = datetime.datetime.now()
later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)
```

In [80]:
#Your code here; use your k-folds function to evaluate the baseline model.
k_folds(X_train, y_train, model)

3626/3626 [==============================] - 0s 36us/step
Finished Fold 0 in 205.54 seconds
3625/3625 [==============================] - 0s 49us/step
Finished Fold 1 in 238.89 seconds
3625/3625 [==============================] - 0s 37us/step
Finished Fold 2 in 227.72 seconds
3625/3625 [==============================] - 0s 37us/step
Finished Fold 3 in 199.94 seconds
3625/3625 [==============================] - 0s 36us/step
Finished Fold 4 in 200.56 seconds
3625/3625 [==============================] - 0s 36us/step
Finished Fold 5 in 200.70 seconds
3625/3625 [==============================] - 0s 37us/step
Finished Fold 6 in 199.83 seconds
3625/3625 [==============================] - 0s 37us/step
Finished Fold 7 in 200.04 seconds
3625/3625 [==============================] - 0s 38us/step
Finished Fold 8 in 199.85 seconds
3625/3625 [==============================] - 0s 48us/step
Finished Fold 9 in 209.80 seconds
Mean MSE: 0.18	 Std MSE: 0.00
Mean Val: 0.19	 Std Val: 0.02


## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.  

For example, here are some timing notes on potential experiments run on a Macbook Pro 3.1 GHz Intel Core i5 with 16gb of RAM:

* Using our 10 fold cross validation methodology, a 5-layer neural network with 10 units per hidden layer and 100 epochs took approximately 15 minutes to train and validate  

* Using our 10 fold cross validation methodology, a 5-layer neural network with 25 units per hidden layer and 100 epochs took approximately 25 minutes to train and validate  

* Using our 10 fold cross validation methodology, a 5-layer neural network with 10 units per hidden layer and 250 epochs took approximately 45 minutes to train and validate


In [ ]:
#Your code here; try some methods to overfit your network

In [ ]:
#Your code here; try some methods to overfit your network

In [ ]:
#Your code here; try some methods to overfit your network

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least 2 different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting.

Notes: 

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your finalized model once and for all. Fit the model using all of the training and validation data using the architecture and hyperparameters that were most effective in your expirements above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modelling process)!

In [ ]:
#Your code here; final model training on entire training set followed by evaluation on hold-out data

## Additional Resources

https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network
https://www.springboard.com/blog/free-public-data-sets-data-science-project/

## Summary

In this lab, we investigated some data from *The Lending Club* in a complete data science pipeline regarding neural networks. We began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, we implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. From here, we'll begin to investigate other neural network architectures such as CNNs.